In [1]:
import os
import sys
import pickle
import numpy as np
import torch 
from torch import nn
from tqdm import tqdm

import mlx.core as mx

os.environ["CONFIG_PATHS"] = "../configs/self_play_20.yaml"
os.environ["CONFIG_OVERRIDES"] = 'game.moves_directory="../data/moves_20"'
sys.path.append("../src")

from configuration import config
from neural_net import NeuralNet
from neural_net_mlx import NeuralNetMLX
""

Loaded config:  {"development": {"debug_mode": true, "profile": false, "runtime": 0, "display_logs_in_console": false, "output_directory": "data/2024-12-30_23-23-24-rubefaction"}, "logging": {"save_interval": 3600, "mcts_report_fraction": 0, "ucb_report": false, "gpu_evaluation": true, "made_move": true}, "game": {"board_size": 20, "num_moves": 30433, "num_pieces": 21, "moves_directory": "../data/moves_20"}, "architecture": {"gameplay_processes": 6, "coroutines_per_process": 256, "game_flush_threshold": 200}, "training": {"run": true, "network_name": "default", "batch_size": 128, "policy_loss_weight": 0.158, "learning_rate": 0.001, "sample_window": 50000, "samples_per_generation": 10000, "sampling_ratio": 2.0, "minimum_window_size": 10000, "new_data_check_interval": 60}, "networks": {"default": {"main_body_channels": 64, "value_head_channels": 16, "value_head_flat_layer_width": 64, "policy_head_channels": 32, "residual_blocks": 10, "model_path": "", "model_directory": "data/2024-12-30_

''

In [5]:
BOARD_SIZE = config()["game"]["board_size"]

In [3]:
import time

def time_per_eval(num_evaluations, batch_size, dtype, model):
    num_batches = num_evaluations // batch_size
    num_evaluations = num_batches * batch_size

    random_arrays = np.random.random((num_batches, batch_size, 4, BOARD_SIZE, BOARD_SIZE))
    print("Starting...")

    start = time.perf_counter()
    for i in range(num_batches):
        if isinstance(model, NeuralNet):
            boards = torch.from_numpy(random_arrays[i]).to(device="mps", dtype=dtype)
            result = model(boards)
        else:
            boards = mx.array(random_arrays[i], dtype=dtype)
            result = model(boards)
            
        if result[0][56][2] > 0:
            pass

    elapsed = time.perf_counter() - start

    return elapsed / (num_batches * batch_size)

In [4]:
torch.save(model_torch.state_dict(), "/Users/shivamsarodia/Dev/blockus/data/2024-12-30_23-23-24-rubefaction/models/0.pt")

In [2]:
batch_size = 256

model_torch = NeuralNet(config()["networks"]["default"])
model_parameters = filter(lambda p: p.requires_grad, model_torch.parameters())
print("Model size:", sum(p.numel() for p in model_parameters))

model_torch.half()
model_torch = model_torch.to("mps")

# Warm up
time_per_eval(
    num_evaluations=batch_size,
    batch_size=batch_size,
    model=model_torch,
    precision=precision
)

time_per_eval(
    num_evaluations=batch_size * 500,
    batch_size=batch_size,
    model=model_torch,
    precision=precision,
)

Model size: 390728197


NameError: name 'time_per_eval' is not defined

In [5]:
precision = 16
batch_size = 256

from mlx.utils import tree_flatten

model_mlx = NeuralNetMLX(config()["networks"]["default"])
model_mlx.eval()

total = 0
for name, p in tree_flatten(model_mlx.parameters()):
    total += np.prod(p.shape)
print("Total parameters:", total)


if precision == 16:
    model_mlx.set_dtype(mx.bfloat16)

compiled_model = mx.compile(model_mlx)

# Warm up
time_per_eval(
    num_evaluations=batch_size,
    batch_size=batch_size,
    model=compiled_model,
    precision=precision
)

time_per_eval(
    num_evaluations=batch_size * 500,
    batch_size=batch_size,
    model=compiled_model,
    precision=precision
)

Total parameters: 40603245
Starting...
Starting...
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/shivamsarodia/.pyenv/versions/3.11.3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/np/v76cnj490z525wk67wqh68dc0000gn/T/ipykernel_8733/3669479805.py", line 28, in <module>
    time_per_eval(
  File "/var/folders/np/v76cnj490z525wk67wqh68dc0000gn/T/ipykernel_8733/8514060.py", line -1, in time_per_eval
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/shivamsarodia/.pyenv/versions/3.11.3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shivamsarodia/.pyenv/versions/3.11.3/lib/python3.11/site-packages/IPython/core/ultratb.py", line 1396, in structured_traceback
    return FormattedTB

In [224]:
random_array = np.random.random((128, 4, BOARD_SIZE, BOARD_SIZE))

In [110]:
compiled_model(mx.array(random_array))[0][56][2]

array(-0.0199862, dtype=float32)

In [114]:
model_mlx(mx.array(random_array))[0][56][2]

array(-0.0199861, dtype=float32)

In [115]:
model_mlx.parameters()["convolutional_block"]["layers"][0]["weight"].dtype

mlx.core.float32

In [26]:
model_mlx(torch.randn(1, 4, BOARD_SIZE, BOARD_SIZE))

TypeError: 'NeuralNetMLX' object is not callable

In [22]:
model_32 = NeuralNet(config()["networks"]["default"])
model_32.load_state_dict(
    torch.load("/Users/shivamsarodia/Dev/blockus/data/2024-11-23_00-37-50-doublehandedness/models/002190002.pt", weights_only=True)
)
model_16 = NeuralNet(config()["networks"]["default"])
model_16.load_state_dict(
    torch.load("/Users/shivamsarodia/Dev/blockus/data/2024-11-23_00-37-50-doublehandedness/models/002190002.pt", weights_only=True)
)
model_16.half()

params_32 = []
for param in model_32.parameters():
    params_32.append(param)

params_16 = []
for param in model_16.parameters():
    params_16.append(param)

most_positive = 0
most_negative = 0
biggest_absolute = 0
biggest_reciprocal = 0
for p_32 in params_32:
    most_positive = max(most_positive, torch.max(p_32))
    most_negative = min(most_negative, torch.min(p_32))
    biggest_absolute = max(biggest_absolute, torch.max(torch.abs(p_32)))
    biggest_reciprocal = max(biggest_reciprocal, torch.max(1 / torch.abs(p_32)))    

print(most_positive, most_negative, biggest_absolute, biggest_reciprocal)

biggest_error = 0
for p_32, p_16 in zip(params_32, params_16):
    error = torch.max( torch.abs( (p_32 - p_16) ) )
    if error > biggest_error:
        biggest_error = error
        print(error)

tensor(2.2047, grad_fn=<MaxBackward1>) tensor(-14.3681, grad_fn=<MinBackward1>) tensor(14.3681, grad_fn=<MaxBackward1>) tensor(81517992., grad_fn=<MaxBackward1>)
tensor(0.0002, grad_fn=<MaxBackward1>)
tensor(0.0005, grad_fn=<MaxBackward1>)
tensor(0.0005, grad_fn=<MaxBackward1>)
tensor(0.0039, grad_fn=<MaxBackward1>)


In [35]:
input = torch.zeros(1, 4, BOARD_SIZE, BOARD_SIZE)

values_32, policy_32 = model_32(input)
values_16, policy_16 = model_16(input.half())

In [39]:
torch.max(torch.abs((policy_32 - policy_16) / policy_32))

tensor(0.0030, grad_fn=<MaxBackward1>)

In [42]:
torch.softmax(values_32, dim=1)

tensor([[0.8424, 0.0459, 0.1071, 0.0045]], grad_fn=<SoftmaxBackward0>)

In [43]:
torch.softmax(values_16, dim=1)

tensor([[0.8423, 0.0461, 0.1072, 0.0045]], dtype=torch.float16,
       grad_fn=<SoftmaxBackward0>)

In [9]:
max(torch.max(p) for p in params_32)

tensor(2.2047, grad_fn=<MaxBackward1>)

In [12]:
min(torch.min(p) for p in params_32)

tensor(-14.3681, grad_fn=<MinBackward1>)

In [10]:
max(torch.max(p) for p in params_16)

tensor(2.2051, dtype=torch.float16, grad_fn=<MaxBackward1>)

In [11]:
min(torch.min(p) for p in params_16)

tensor(-14.3672, dtype=torch.float16, grad_fn=<MinBackward1>)

In [16]:
for i in range(len(params)):
    print(params[i].dtype, half_params[i].dtype)
    print(params[i] == half_params[i])

torch.float16 torch.float16
tensor([[[[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]]],


        [[[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]]],


        [[[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, True, True],
          [True, True, True]],

         [[True, True, True],
          [True, T